In [1]:
from keras.models import load_model
import cv2
import numpy as np
import tkinter
from tkinter import messagebox
import smtplib
import time
from threading import Thread
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from email.mime.multipart import MIMEMultipart

In [2]:
root = tkinter.Tk()
root.withdraw()

''

In [3]:
def send_email(label):
    SUBJECT = "Violation"   
    TEXT = "One Visitor violated Face Mask Policy. See in the camera to recognize user. A Person has been detected without a face mask in the Hotel Lobby Area 9. Please Alert the authorities."
    while True:
        if label == 1:
            message = MIMEMultipart()
            message['SUBJECT'] = 'SUBJECT'
            message['From'] = '1gobinathal8@gmail.com'
            message['To'] = '1gobinathal8@gmail.com'
            message.attach(MIMEText('Subject: {}\n\n{}'.format(SUBJECT, TEXT)))
            with open("filename.jpg", 'rb') as f:
                image = MIMEImage(f.read())
            message.attach(image)
            mail = smtplib.SMTP('smtp.gmail.com', 587)
            mail.ehlo()
            mail.starttls()
            mail.login('1gobinathal8@gmail.com','Gobinath 22@')
            mail.sendmail('1gobinathal8@gmail.com','1gobinathal8@gmail.com',message.as_string())
            mail.close
            print("message sent")
            time.sleep(10)

In [ ]:

model = load_model('face_mask_detection_alert_system.h5')

face_det_classifier=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

vid_source=cv2.VideoCapture(0)

text_dict={0:'Mask ON',1:'No Mask'}
rect_color_dict={0:(0,255,0),1:(0,0,255)}


i = 0
while(True):

    ret, img = vid_source.read()
    cv2.imwrite("filename.jpg",img)
    grayscale_img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_det_classifier.detectMultiScale(grayscale_img,1.3,5)  

    for (x,y,w,h) in faces:
    
        face_img = grayscale_img[y:y+w,x:x+w]
        resized_img = cv2.resize(face_img,(112,112))
        normalized_img = resized_img/255.0
        reshaped_img = np.reshape(normalized_img,(1,112,112,1))
        result=model.predict(reshaped_img)
        pred = model.predict_classes(reshaped_img)
        label=np.argmax(result,axis=1)[0]
        cv2.rectangle(img,(x,y),(x+w,y+h),rect_color_dict[label],2)
        cv2.rectangle(img,(x,y-40),(x+w,y),rect_color_dict[label],-1)
        cv2.putText(img, text_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(0,0,0),2) 
        if i == 0:
            t = Thread(target = send_email, args = (label, ))
            t.start()
        i += 1
        
    cv2.imshow('LIVE Video Feed',img)
    key=cv2.waitKey(1)
    
cv2.destroyAllWindows()
source.release()

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
message sent
